In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os

os.chdir('/content/drive/MyDrive/Umbrella to Secure Access Migration/Project Data')

os.listdir()

Mounted at /content/drive


['SIG Pipeline FY25Q3-FY26Q1 (Pulled Jan 23, 2025) copy.xlsx',
 'FY25 SBG Unit Cost Analysis.pptx',
 'acct_details_Full Data_data.xlsx',
 'inventory.json',
 '250708 Migration Model.ipynb',
 '250717 Migration Model.ipynb',
 'FY26 Q1 Q2 AWS POPs to Enable.xlsx',
 'Migration Capacity Analysis.ipynb',
 'Available AWS POPs.xlsx',
 'worldcities.xlsx',
 'Top src city traffic Kentik.xlsx',
 'Kentik Geocode.ipynb',
 'Geocoded Kentik Data.xlsx',
 'Aug 3 Migration Optimization (CITY-LEVEL).ipynb',
 'Infrastructure Priority List Builder.ipynb',
 'Current and Planned SA Coverage.xlsx',
 'Merged Data for Migration.xlsx',
 'Optimized_DCv2_Sites.xlsx',
 'Clustered_Optimized_DC_Map_With_Legend.html',
 'Optimized_DC_Map_With_Legend_No_Clusters.html',
 'Optimized DC Expansion Timeline.xlsx',
 'Untitled0.ipynb',
 'Shiv Gupta Umbrella to Secure Access Geographic Optimization Model Final.ipynb']

In [2]:
!pip install pulp
!pip install branca
import pandas as pd
from google.colab import files

# Load Excel files (assumes they're already in the working directory or Google Drive is mounted)
merged_df = pd.read_excel("Merged Data for Migration.xlsx")
kentik_df = pd.read_excel("Geocoded Kentik Data.xlsx")
coverage_df = pd.read_excel("Current and Planned SA Coverage.xlsx")
aws_df = pd.read_excel("Available AWS POPs.xlsx")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.9 MB/s eta 0:00:00


In [3]:
# Keep only SIG customers with valid seat counts
sig_customers_df = (
    merged_df[
        (merged_df["Product"].str.upper() == "SIG") &
        (merged_df["SIG Adjusted Seatcount"] > 0)
    ][["Cx Customer Bu Name", "Cx Cust Prim Country Cd", "SIG Adjusted Seatcount", "Pipeline TCV"]]
    .copy()
)

# Keep only cities with average traffic ≥ 15,000 packets/s and valid coordinates
kentik_priority_cities = (
    kentik_df[kentik_df["Average packets/s"] >= 15000][[
        "Src City", "Average packets/s", "lat", "lng", "country", "iso2"
    ]]
    .dropna(subset=["lat", "lng"])
    .drop_duplicates(subset=["Src City", "country"])
    .rename(columns={"lat": "Latitude", "lng": "Longitude"})
    .reset_index(drop=True)
    .copy()
)

# Clean latitude and longitude columns
kentik_df["lat"] = pd.to_numeric(kentik_df["lat"], errors="coerce")
kentik_df["lng"] = pd.to_numeric(kentik_df["lng"], errors="coerce")

# Drop rows where lat/lng couldn't be converted
kentik_cleaned = kentik_df.dropna(subset=["lat", "lng"]).copy()

# Optional: rename for consistency with other parts of your code
kentik_cleaned.rename(columns={"lat": "Latitude", "lng": "Longitude"}, inplace=True)


# Rename for consistency
coverage_df.rename(columns={"Data Center(s)": "Site"}, inplace=True)

aws_pops_df = (
    aws_df.rename(columns={"Data Center(s)": "Site"})
    [["Site", "Country", "Latitude", "Longitude"]]
    .dropna()
    .copy()
)


In [4]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate great-circle distance (km) between two points on Earth.
    """
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

covered_city_indices = []

for idx, city in kentik_priority_cities.iterrows():
    city_lat = city["Latitude"]
    city_lon = city["Longitude"]

    distances = haversine(
        city_lat, city_lon,
        coverage_df["Latitude"].values,
        coverage_df["Longitude"].values
    )

    if np.any(distances <= 500):
        covered_city_indices.append(idx)

# Covered and uncovered city dataframes
covered_cities_df = kentik_priority_cities.loc[covered_city_indices].copy()
uncovered_cities_df = kentik_priority_cities.drop(index=covered_city_indices).copy()

In [5]:
print(f"Total high-priority cities: {len(kentik_priority_cities)}")
print(f"Cities already covered: {len(covered_cities_df)}")
print(f"Cities needing new DCv2: {len(uncovered_cities_df)}")


Total high-priority cities: 158
Cities already covered: 122
Cities needing new DCv2: 36


In [6]:
# Create a new column to indicate AWS fallback coverage
uncovered_cities_df["Covered by AWS Fallback"] = False

for idx, city in uncovered_cities_df.iterrows():
    city_lat = city["Latitude"]
    city_lon = city["Longitude"]

    distances = haversine(
        city_lat, city_lon,
        aws_pops_df["Latitude"].values,
        aws_pops_df["Longitude"].values
    )

    if np.any(distances <= 500):
        uncovered_cities_df.at[idx, "Covered by AWS Fallback"] = True

print("Uncovered cities with AWS fallback coverage:",
      uncovered_cities_df["Covered by AWS Fallback"].sum())


Uncovered cities with AWS fallback coverage: 3


In [7]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary
from collections import defaultdict

# === Step 1: Build Coverage Matrix ===
coverage_matrix = defaultdict(list)

for i, city in uncovered_cities_df.iterrows():
    city_name = city["Src City"]
    city_lat, city_lon = city["Latitude"], city["Longitude"]
    for j, candidate in uncovered_cities_df.iterrows():
        cand_name = candidate["Src City"]
        cand_lat, cand_lon = candidate["Latitude"], candidate["Longitude"]
        if haversine(city_lat, city_lon, cand_lat, cand_lon) <= 500:
            coverage_matrix[city_name].append(cand_name)

# === Step 2: Define Optimization Model ===
model = LpProblem("Minimize_New_DCv2s", LpMinimize)

# Decision Variables: x_j = 1 if we place a DCv2 in candidate city j
x = LpVariable.dicts("Build_DC", uncovered_cities_df["Src City"], cat=LpBinary)

# Objective: minimize total DCv2s
model += lpSum(x[j] for j in uncovered_cities_df["Src City"]), "Minimize_Total_Sites"

# Constraints: Every uncovered city must be within 500km of a selected site
for city in uncovered_cities_df["Src City"]:
    if coverage_matrix[city]:  # only if there are possible sites
        model += lpSum(x[j] for j in coverage_matrix[city]) >= 1, f"Cover_{city}"

# === Step 3: Solve ===
model.solve()


1

In [8]:
# Get selected cities where the model chose to place a new DC
selected_dc_cities = [city for city in uncovered_cities_df["Src City"] if x[city].varValue == 1.0]

# Filter the full city metadata (lat/lon, country, etc.)
optimized_dc_df = uncovered_cities_df[uncovered_cities_df["Src City"].isin(selected_dc_cities)].copy()

print(f"✅ {len(optimized_dc_df)} optimized DCv2 sites selected.")


✅ 34 optimized DCv2 sites selected.


In [9]:
optimized_dc_df.to_excel("Optimized_DCv2_Sites.xlsx", index=False)

In [10]:
import folium
from folium.plugins import MarkerCluster

# === Initialize map ===
map_center = [optimized_dc_df["Latitude"].mean(), optimized_dc_df["Longitude"].mean()]
base_map = folium.Map(location=map_center, zoom_start=2, tiles="cartodb positron")

# === Create separate marker clusters ===
optimized_standard_cluster = MarkerCluster(name="Optimized DCv2 (Standard)").add_to(base_map)
optimized_fallback_cluster = MarkerCluster(name="Optimized DCv2 (AWS Fallback)").add_to(base_map)
existing_cluster = MarkerCluster(name="Existing Infrastructure").add_to(base_map)
traffic_cluster = MarkerCluster(name="Traffic Source Cities").add_to(base_map)

# === Add Optimized DCv2 Markers + 500km Radius, split by AWS fallback ===
for _, row in optimized_dc_df.iterrows():
    city = row["Src City"]
    country = row["country"]
    lat, lon = row["Latitude"], row["Longitude"]
    aws_flag = row.get("Covered by AWS Fallback", False)

    color = "red" if aws_flag else "green"
    popup_text = f"<b>{city}</b><br>{country}<br>AWS Fallback: {aws_flag}"

    # Select cluster based on fallback status
    target_cluster = optimized_fallback_cluster if aws_flag else optimized_standard_cluster

    # Marker
    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        popup=folium.Popup(popup_text, max_width=250)
    ).add_to(target_cluster)

    # Radius circle
    folium.Circle(
        location=[lat, lon],
        radius=500000,
        color=color,
        fill=False,
        weight=2,
        dash_array='5, 10'
    ).add_to(target_cluster)


# === Add Existing Infrastructure Markers + 500km Radius ===
for _, row in coverage_df.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    site_type = str(row.get("DC Type", "Unknown"))
    name = row.get("Site", "Unnamed Site")
    status = row.get("Status", "Used")

    color = "blue" if "Cisco Edge v2" in site_type else "orange"
    popup = f"<b>{name}</b><br>Type: {site_type}<br>Status: {status}"

    # Marker
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup, max_width=250)
    ).add_to(existing_cluster)

    # Radius circle (500km)
    folium.Circle(
        location=[lat, lon],
        radius=500000,
        color=color,
        fill=False,
        weight=2,
        dash_array='5, 10'
    ).add_to(existing_cluster)

# === Add Kentik Traffic Source Cities ===
for _, row in kentik_cleaned.iterrows():
    city = row["Src City"]
    packets = row["Average packets/s"]
    lat, lon = row["Latitude"], row["Longitude"]

    folium.CircleMarker(
        location=[lat, lon],
        radius=3,
        color="darkred",
        fill=True,
        fill_color="darkred",
        fill_opacity=0.6,
        popup=folium.Popup(f"<b>{city}</b><br>Avg p/s: {packets:,.0f}", max_width=250)
    ).add_to(traffic_cluster)

# === Add Layer Control ===
folium.LayerControl().add_to(base_map)

from branca.element import Template, MacroElement

legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 30px; left: 30px; width: 260px; z-index: 9999;
    background-color: white; border:2px solid grey; border-radius:6px;
    padding: 10px; font-size:14px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
">
    <b>🗺 Map Legend</b><br>
    <i style="background: green; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    Optimized DCv2 Site<br>
    <i style="background: red; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    Optimized DCv2 Site (AWS fallback)<br>
    <i style="background: blue; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    Existing Cisco DCv2<br>
    <i style="background: orange; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    AWS POP<br>
    <i style="background: darkred; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    SIG Traffic Source City<br>
    <i style="border: 1px dashed black; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i>
    500km Radius Coverage
</div>
{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
base_map.get_root().add_child(legend)



# === Show Map ===
base_map


In [11]:
base_map.save("Clustered_Optimized_DC_Map_With_Legend.html")
files.download("Clustered_Optimized_DC_Map_With_Legend.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import folium

# === Initialize map ===
map_center = [optimized_dc_df["Latitude"].mean(), optimized_dc_df["Longitude"].mean()]
base_map = folium.Map(location=map_center, zoom_start=2, tiles="cartodb positron")

# === Create feature groups instead of clusters ===
optimized_standard_group = folium.FeatureGroup(name="Optimized DCv2 (Standard)").add_to(base_map)
optimized_fallback_group = folium.FeatureGroup(name="Optimized DCv2 (AWS Fallback)").add_to(base_map)
existing_group = folium.FeatureGroup(name="Existing Infrastructure").add_to(base_map)
traffic_group = folium.FeatureGroup(name="Traffic Source Cities").add_to(base_map)

# === Add Optimized DCv2 Markers + 500km Radius, split by AWS fallback ===
for _, row in optimized_dc_df.iterrows():
    city = row["Src City"]
    country = row["country"]
    lat, lon = row["Latitude"], row["Longitude"]
    aws_flag = row.get("Covered by AWS Fallback", False)

    color = "red" if aws_flag else "green"
    popup_text = f"<b>{city}</b><br>{country}<br>AWS Fallback: {aws_flag}"
    group = optimized_fallback_group if aws_flag else optimized_standard_group

    # Marker
    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        popup=folium.Popup(popup_text, max_width=250)
    ).add_to(group)

    # Radius circle
    folium.Circle(
        location=[lat, lon],
        radius=500000,
        color=color,
        fill=False,
        weight=2,
        dash_array='5, 10'
    ).add_to(group)

# === Add Existing Infrastructure Markers + 500km Radius ===
for _, row in coverage_df.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    site_type = str(row.get("DC Type", "Unknown"))
    name = row.get("Site", "Unnamed Site")
    status = row.get("Status", "Used")

    color = "blue" if "Cisco Edge v2" in site_type else "orange"
    popup = f"<b>{name}</b><br>Type: {site_type}<br>Status: {status}"

    # Marker
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup, max_width=250)
    ).add_to(existing_group)

    # Radius circle
    folium.Circle(
        location=[lat, lon],
        radius=500000,
        color=color,
        fill=False,
        weight=2,
        dash_array='5, 10'
    ).add_to(existing_group)

# === Add Kentik Traffic Source Cities ===
for _, row in kentik_cleaned.iterrows():
    city = row["Src City"]
    packets = row["Average packets/s"]
    lat, lon = row["Latitude"], row["Longitude"]

    folium.CircleMarker(
        location=[lat, lon],
        radius=3,
        color="darkred",
        fill=True,
        fill_color="darkred",
        fill_opacity=0.6,
        popup=folium.Popup(f"<b>{city}</b><br>Avg p/s: {packets:,.0f}", max_width=250)
    ).add_to(traffic_group)

# === Add Layer Control ===
folium.LayerControl(collapsed=False).add_to(base_map)

# === Add Legend (same as before) ===
from branca.element import Template, MacroElement

legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 30px; left: 30px; width: 260px; z-index: 9999;
    background-color: white; border:2px solid grey; border-radius:6px;
    padding: 10px; font-size:14px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
">
    <b>🗺 Map Legend</b><br>
    <i style="background: green; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    Optimized DCv2 Site<br>
    <i style="background: red; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    Optimized DCv2 Site (AWS fallback)<br>
    <i style="background: blue; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    Existing Cisco DCv2<br>
    <i style="background: orange; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    AWS POP<br>
    <i style="background: darkred; width: 12px; height: 12px; display: inline-block; border-radius: 50%; margin-right: 5px;"></i>
    SIG Traffic Source City<br>
    <i style="border: 1px dashed black; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i>
    500km Radius Coverage
</div>
{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
base_map.get_root().add_child(legend)

# === Show Map ===
base_map


In [13]:
base_map.save("Optimized_DC_Map_With_Legend_No_Clusters.html")
files.download("Optimized_DC_Map_With_Legend_No_Clusters.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# Standardize column names
merged_df = merged_df.rename(columns={
    "Cx Cust Prim Country Cd": "Country Code",
    "SIG Adjusted Seatcount": "SIG Seats",
    "Pipeline TCV": "TCV",
    "Cx Customer Bu Id": "Customer ID",
    "Contract End Date": "Contract End Date",
})

# Step 1: Convert Contract End Date to datetime
merged_df["Contract End Date"] = pd.to_datetime(merged_df["Contract End Date"], errors="coerce")

# Step 2: Create Fiscal Quarter Column
def get_fiscal_quarter(date):
    if pd.isnull(date):
        return None
    month = date.month
    year = date.year

    if month in [8, 9, 10]:   # Aug, Sep, Oct → Q1 FY+1
        fiscal_quarter = "Q1"
        fiscal_year = year + 1
    elif month in [11, 12]:   # Nov, Dec → Q2 FY+1
        fiscal_quarter = "Q2"
        fiscal_year = year + 1
    elif month == 1:          # Jan → Q2 FY
        fiscal_quarter = "Q2"
        fiscal_year = year
    elif month in [2, 3, 4]:  # Feb, Mar, Apr → Q3 FY
        fiscal_quarter = "Q3"
        fiscal_year = year
    elif month in [5, 6, 7]:  # May, Jun, Jul → Q4 FY
        fiscal_quarter = "Q4"
        fiscal_year = year
    else:
        return None

    return f"{fiscal_quarter} FY{fiscal_year}"

merged_df["Fiscal Quarter"] = merged_df["Contract End Date"].apply(get_fiscal_quarter)

# Step 3: Filter for SIG customers only
sig_df = merged_df[merged_df["Product"].str.upper() == "SIG"]

# Step 4: Filter SIG customers to only countries with optimized DCv2 sites
optimized_countries = optimized_dc_df["iso2"].dropna().unique()
sig_df = sig_df[sig_df["Country Code"].isin(optimized_countries)].copy()

# Step 5: Filter to quarters Q3 FY2026 and beyond
def quarter_sort_key(fq):
    q_part, fy_part = fq.strip().split()
    q_num = int(q_part[1])
    fy_num = int(fy_part.replace("FY", ""))
    return fy_num * 10 + q_num

sig_df["Quarter_Sort_Key"] = sig_df["Fiscal Quarter"].apply(quarter_sort_key)
sig_df = sig_df[sig_df["Quarter_Sort_Key"] >= 20263]

# Step 6: Group by Fiscal Quarter and Country
priority_ranking = (
    sig_df.dropna(subset=["Fiscal Quarter", "Country Code"])
    .groupby(["Fiscal Quarter", "Country Code"])
    .agg(
        Total_SIG_Seats=("SIG Seats", "sum"),
        Total_TCV=("TCV", "sum"),
        Customer_Count=("Customer ID", "nunique")
    )
    .reset_index()
)

# Step 7: Final sort
priority_ranking["Quarter_Sort_Key"] = priority_ranking["Fiscal Quarter"].apply(quarter_sort_key)
priority_ranking = priority_ranking.sort_values(
    by=["Quarter_Sort_Key", "Total_TCV"],
    ascending=[True, False]
).drop(columns="Quarter_Sort_Key").reset_index(drop=True)

# Format TCV as USD (e.g., $3,400,000)
priority_ranking["Total_TCV"] = priority_ranking["Total_TCV"].apply(lambda x: f"${x:,.0f}")

# Preview result
priority_ranking


,Fiscal Quarter,Country Code,Total_SIG_Seats,Total_TCV,Customer_Count
0,Q3 FY2026,US,178803.0,"$19,979,607",184
1,Q3 FY2026,NG,4000.0,"$432,750",1
2,Q3 FY2026,AU,9050.0,"$367,953",9
3,Q3 FY2026,DO,3567.0,"$267,148",2
4,Q3 FY2026,NZ,0.0,"$239,040",1
5,Q3 FY2026,MX,2298.0,"$220,842",7
6,Q3 FY2026,TH,119.0,"$87,167",1
7,Q3 FY2026,TR,1250.0,"$82,889",2
8,Q3 FY2026,IN,1865.0,"$44,895",6
9,Q3 FY2026,ZA,2566.0,"$36,731",4


In [16]:
# Standardize column names
merged_df = merged_df.rename(columns={
    "Cx Cust Prim Country Cd": "Country Code",
    "SIG Adjusted Seatcount": "SIG Seats",
    "Pipeline TCV": "TCV",
    "Cx Customer Bu Id": "Customer ID",
    "Contract End Date": "Contract End Date",
})

# Step 1: Convert Contract End Date to datetime
merged_df["Contract End Date"] = pd.to_datetime(merged_df["Contract End Date"], errors="coerce")

# Step 2: Create Fiscal Quarter Column
def get_fiscal_quarter(date):
    if pd.isnull(date):
        return None
    month = date.month
    year = date.year

    if month in [8, 9, 10]:   # Aug, Sep, Oct → Q1 FY+1
        fiscal_quarter = "Q1"
        fiscal_year = year + 1
    elif month in [11, 12]:   # Nov, Dec → Q2 FY+1
        fiscal_quarter = "Q2"
        fiscal_year = year + 1
    elif month == 1:          # Jan → Q2 FY
        fiscal_quarter = "Q2"
        fiscal_year = year
    elif month in [2, 3, 4]:  # Feb, Mar, Apr → Q3 FY
        fiscal_quarter = "Q3"
        fiscal_year = year
    elif month in [5, 6, 7]:  # May, Jun, Jul → Q4 FY
        fiscal_quarter = "Q4"
        fiscal_year = year
    else:
        return None

    return f"{fiscal_quarter} FY{fiscal_year}"

merged_df["Fiscal Quarter"] = merged_df["Contract End Date"].apply(get_fiscal_quarter)

# Step 3: Filter for SIG customers only
sig_df = merged_df[merged_df["Product"].str.upper() == "SIG"]

# Step 4: Filter SIG customers to only countries with optimized DCv2 sites
optimized_countries = optimized_dc_df["iso2"].dropna().unique()
sig_df = sig_df[sig_df["Country Code"].isin(optimized_countries)].copy()

# Step 5: Filter to quarters Q3 FY2026 and beyond
def quarter_sort_key(fq):
    q_part, fy_part = fq.strip().split()
    q_num = int(q_part[1])
    fy_num = int(fy_part.replace("FY", ""))
    return fy_num * 10 + q_num

sig_df["Quarter_Sort_Key"] = sig_df["Fiscal Quarter"].apply(quarter_sort_key)
sig_df = sig_df[sig_df["Quarter_Sort_Key"] >= 20263]

# STEP 6: Group by Fiscal Quarter and Country
priority_ranking = (
    sig_df.dropna(subset=["Fiscal Quarter", "Country Code"])
    .groupby(["Fiscal Quarter", "Country Code"])
    .agg(
        Total_SIG_Seats=("SIG Seats", "sum"),
        Total_TCV=("TCV", "sum"),
        Customer_Count=("Customer ID", "nunique")
    )
    .reset_index()
)

# STEP 7: Add numeric sort key for future filtering
priority_ranking["Quarter_Sort_Key"] = priority_ranking["Fiscal Quarter"].apply(quarter_sort_key)

# STEP 8: Compute Long-Term TCV Capture (rolling sum by country)
priority_ranking["Longterm_TCV"] = 0.0

# For each country, sort by quarter and compute rolling sum
for country in priority_ranking["Country Code"].unique():
    country_mask = priority_ranking["Country Code"] == country
    country_df = priority_ranking[country_mask].copy()
    country_df = country_df.sort_values("Quarter_Sort_Key", ascending=True)

    # Cumulative sum of Total_TCV in reverse (future quarters)
    cum_tcv = country_df["Total_TCV"][::-1].cumsum()[::-1]

    # Assign values back
    priority_ranking.loc[country_mask, "Longterm_TCV"] = cum_tcv.values

# STEP 9: Final sort and formatting
priority_ranking = priority_ranking.sort_values(
    by=["Quarter_Sort_Key", "Longterm_TCV"],
    ascending=[True, False]
).reset_index(drop=True)

# Format currency values
priority_ranking["Total_TCV"] = priority_ranking["Total_TCV"].apply(lambda x: f"${x:,.0f}")
priority_ranking["Longterm_TCV"] = priority_ranking["Longterm_TCV"].apply(lambda x: f"${x:,.0f}")

# Optional: Drop helper column
priority_ranking = priority_ranking.drop(columns="Quarter_Sort_Key")

# Preview
priority_ranking



,Fiscal Quarter,Country Code,Total_SIG_Seats,Total_TCV,Customer_Count,Longterm_TCV
0,Q3 FY2026,US,178803.0,"$19,979,607",184,"$4,865,197"
1,Q3 FY2026,ZA,2566.0,"$36,731",4,"$2,125,296"
2,Q3 FY2026,NG,4000.0,"$432,750",1,"$432,750"
3,Q3 FY2026,DO,3567.0,"$267,148",2,"$314,668"
4,Q3 FY2026,NZ,0.0,"$239,040",1,"$248,587"
5,Q3 FY2026,AU,9050.0,"$367,953",9,"$116,697"
6,Q3 FY2026,MX,2298.0,"$220,842",7,"$87,399"
7,Q3 FY2026,TH,119.0,"$87,167",1,"$87,167"
8,Q3 FY2026,TR,1250.0,"$82,889",2,"$86,039"
9,Q3 FY2026,IN,1865.0,"$44,895",6,"$47,601"


In [17]:
priority_ranking.to_excel("Optimized DC Expansion Timeline.xlsx", index=False)

In [18]:
us_tcv_sum = priority_ranking[priority_ranking["Country Code"] == "US"]["Longterm_TCV"].str.replace(r'[$,]', '', regex=True).astype(float).sum()
print(f"Total Longterm_TCV for US: ${us_tcv_sum:,.0f}")

Total Longterm_TCV for US: $101,855,847


In [19]:
cities_to_sum = [
    "Budapest", "Copenhagen", "Lisbon", "Moscow", "Athens", "Tel Aviv",
    "Istanbul", "Zagreb", "Vienna", "Sofia", "Bucharest", "Oslo",
    "Kongens Lyngby", "Tranbjerg", "Hirtshals", "Aalborg"
]

filtered_kentik_df = kentik_df[kentik_df["Src City"].isin(cities_to_sum)]

sum_packets = filtered_kentik_df["Average packets/s"].sum()

print(f"The sum of Average packets/s for the specified cities is: {sum_packets:,.0f}")

The sum of Average packets/s for the specified cities is: 308,140


In [20]:
kentik_df = pd.read_excel("Geocoded Kentik Data.xlsx")

cities_to_sum = [
    "Budapest", "Prague","Copenhagen", "Lisbon", "Moscow", "Athens", "Tel Aviv",
    "Istanbul", "Zagreb", "Vienna", "Sofia", "Bucharest", "Oslo",
    "Kongens Lyngby", "Tranbjerg", "Hirtshals", "Aalborg"
]

filtered_kentik_df = kentik_df[kentik_df["Src City"].isin(cities_to_sum)]

sum_packets = filtered_kentik_df["Average packets/s"].sum()

print(f"The sum of Average packets/s for the specified cities is: {sum_packets:,.0f}")

The sum of Average packets/s for the specified cities is: 327,820


In [21]:
# Get country codes for the cities in cities_to_sum
city_country_codes = kentik_df[kentik_df["Src City"].isin(cities_to_sum)]["iso2"].unique()

# Filter merged_df by these country codes and sum TCV
total_tcv_for_countries = merged_df[merged_df["Country Code"].isin(city_country_codes)]["TCV"].sum()

print(f"The total TCV for the countries of the specified cities is: ${total_tcv_for_countries:,.0f}")

The total TCV for the countries of the specified cities is: $7,468,230
